In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%pip install openai
%pip install ffmpeg-python

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.6 MB/s eta 0:00:00


In [ ]:
#import json
%cd /content/drive/MyDrive/Sanjiv/IMP-OIC-Windowing
import pandas as pd
#import numpy as np
import sys
import os
import shutil
#sys.path.insert(0,'/Users/sanjivjha/IMP-OIC/')
from utils.extractframes import FrameExtractor
import graphene
from STAR_eval.gpt_ask import run_gpt
directory_path = 'STAR_eval/Charades'


/content/drive/MyDrive/Sanjiv/IMP-OIC-Windowing


In [ ]:
QA_ds = pd.read_json('STAR_eval/STAR_val.json')
#QA_ds.head()

In [ ]:
QA = QA_ds.loc[:, ['question_id','question','video_id','start','end','answer', 'choices']]

In [ ]:
#QA = QA_ds_questions.groupby(['question_id','video_id','question','answer']).sum().reset_index()#.to_csv('STAR_eval/question_set.csv')
QA.head()

,question_id,question,video_id,start,end,answer,choices
0,Interaction_T1_13,Which object was tidied up by the person?,6H78U,11.1,19.6,The clothes.,"[{'choice_id': 0, 'choice': 'The closet/cabine..."
1,Interaction_T1_14,Which object was tidied up by the person?,6H78U,15.6,22.7,The clothes.,"[{'choice_id': 0, 'choice': 'The blanket.', 'c..."
2,Interaction_T1_31,Which object was thrown by the person?,RNLTR,7.3,13.5,The clothes.,"[{'choice_id': 0, 'choice': 'The pillow.', 'ch..."
3,Interaction_T1_32,Which object was put down by the person?,RNLTR,11.6,16.4,The shoe.,"[{'choice_id': 0, 'choice': 'The food.', 'choi..."
4,Interaction_T1_40,Which object was tidied up by the person?,VNQTH,2.7,9.2,The closet/cabinet.,"[{'choice_id': 0, 'choice': 'The broom.', 'cho..."


In [ ]:
CLASSES = ['airplane', 'animal', 'arm', 'bag', 'banana', 'basket', 'beach', 'bear', 'bed', 'bench', 'bike',
                'bird', 'board', 'boat', 'book', 'boot', 'bottle', 'bowl', 'box', 'boy', 'branch', 'building',
                'bus', 'cabinet', 'cap', 'car', 'cat', 'chair', 'child', 'clock', 'coat', 'counter', 'cow', 'cup',
                'curtain', 'desk', 'dog', 'door', 'drawer', 'ear', 'elephant', 'engine', 'eye', 'face', 'fence',
                'finger', 'flag', 'flower', 'food', 'fork', 'fruit', 'giraffe', 'girl', 'glass', 'glove', 'guy',
                'hair', 'hand', 'handle', 'hat', 'head', 'helmet', 'hill', 'horse', 'house', 'jacket', 'jean',
                'kid', 'kite', 'lady', 'lamp', 'laptop', 'leaf', 'leg', 'letter', 'light', 'logo', 'man', 'men',
                'motorcycle', 'mountain', 'mouth', 'neck', 'nose', 'number', 'orange', 'pant', 'paper', 'paw',
                'people', 'person', 'phone', 'pillow', 'pizza', 'plane', 'plant', 'plate', 'player', 'pole', 'post',
                'pot', 'racket', 'railing', 'rock', 'roof', 'room', 'screen', 'seat', 'sheep', 'shelf', 'shirt',
                'shoe', 'short', 'sidewalk', 'sign', 'sink', 'skateboard', 'ski', 'skier', 'sneaker', 'snow',
                'sock', 'stand', 'street', 'surfboard', 'table', 'tail', 'tie', 'tile', 'tire', 'toilet', 'towel',
                'tower', 'track', 'train', 'tree', 'truck', 'trunk', 'umbrella', 'vase', 'vegetable', 'vehicle',
                'wave', 'wheel', 'window', 'windshield', 'wing', 'wire', 'woman', 'zebra']

REL_CLASSES = ['above', 'across', 'against', 'along', 'and', 'at', 'attached to', 'behind',
                'belonging to', 'between', 'carrying', 'covered in', 'covering', 'eating', 'flying in', 'for',
                'from', 'growing on', 'hanging from', 'has', 'holding', 'in', 'in front of', 'laying on',
                'looking at', 'lying on', 'made of', 'mounted on', 'near', 'of', 'on', 'on back of', 'over',
                'painted on', 'parked on', 'part of', 'playing', 'riding', 'says', 'sitting on', 'standing on',
                'to', 'under', 'using', 'walking in', 'walking on', 'watching', 'wearing', 'wears', 'with']

words = set(CLASSES+REL_CLASSES)


In [ ]:
import string
index_list = []
for q in QA.loc[:,'question_id'].values:
    questions = QA.query("question_id=='"+q+"'")["question"]
    answer = QA.query("question_id=='"+q+"'")["answer"].values[0].lower().strip('the').translate(str.maketrans('', '', string.punctuation))
    count = 0
    question = ''
    for que in questions:
        question = que.lower().translate(str.maketrans('', '', string.punctuation))

    for i in list(answer.strip().split(' ')) + list(question.strip().split(' ')):
      if i not in  ['a', 'is','by', 'was', 'which', 'man', 'did', 'do', 'woman', 'boy', 'girl', 'person','people','scene','frame','in','to','the', 'of', 'on','with', 'from', 'at', 'and']:
          if i in words:
            index_list.append(QA[QA['question_id']==q].index[0])

print(index_list)
#QA.drop(index_list, inplace = True)
QA = QA[QA.index.isin(index_list)]
QA.info()


[3, 6, 8, 9, 13, 17, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 35, 36, 37, 41, 42, 43, 44, 45, 49, 50, 51, 52, 53, 54, 56, 57, 58, 60, 61, 64, 65, 66, 75, 76, 80, 81, 82, 85, 86, 90, 93, 94, 96, 97, 98, 99, 102, 104, 106, 107, 108, 109, 110, 111, 113, 114, 115, 116, 121, 128, 129, 131, 140, 141, 143, 144, 145, 147, 148, 149, 150, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 175, 176, 177, 178, 179, 180, 182, 184, 185, 187, 189, 190, 192, 195, 199, 200, 201, 202, 203, 204, 205, 206, 209, 210, 212, 217, 219, 220, 221, 222, 223, 229, 232, 236, 238, 239, 240, 243, 244, 245, 246, 247, 248, 254, 255, 256, 257, 258, 259, 261, 265, 266, 267, 269, 271, 272, 274, 277, 278, 290, 291, 292, 296, 297, 298, 303, 307, 308, 311, 312, 313, 318, 319, 321, 323, 326, 327, 328, 329, 330, 339, 345, 349, 350, 351, 352, 361, 363, 364, 367, 372, 374, 375, 376, 377, 379, 380, 382, 383, 386, 387, 390, 391, 394, 395, 400, 401, 402, 403, 405, 406, 407, 408, 41

In [ ]:
QA.to_csv('STAR_eval/question_set.csv')

In [ ]:
import ffmpeg

def segment_video(input_path, output_path, start, end):
    input_file = ffmpeg.input(directory_path+'/'+input_path+'.mp4')
    duration = int(end-start)
    if duration < 10:
      output_file = ffmpeg.output(input_file.trim(start=start, duration = duration).filter('setpts','PTS-STARTPTS'), directory_path+'/trim/'+output_path+'.mp4')
      ffmpeg.run(output_file)
      return True
    else:
      return False

In [ ]:
def run_oic(question_id, video_id, question, choices, start,end):
    TEMP_DIR = "temp"
    CAM_PATH = "cam"
    OUT_DIR = "out"

    for video in os.scandir(directory_path):
        if video.is_file():
            if os.path.splitext(video.name)[0] == video_id:
                print('processing {}'.format(video_id))
                print(video_id, question, choices, start, end)
                video_p = segment_video(os.path.splitext(video.name)[0],question_id+os.path.splitext(video.name)[0],start,end)
                if video_p:
                  ex = FrameExtractor(directory_path+'/trim/'+question_id+video.name, 10, 5)
                  ex.main()
                  g = graphene.Graphene(0.3, 0.6)

                  if not os.path.isdir(OUT_DIR):
                      os.mkdir(OUT_DIR)
                  text = question_id+os.path.splitext(video.name)[0]+'graph2text.txt'
                  img_path = directory_path+'/trim/'+question_id+os.path.splitext(video.name)[0]+'-opencv'
                  #visual = question_id+os.path.splitext(video.name)[0]+'tg.png'
                  #g.run_online(text)


                  g.classify_images_window(img_path, 5)
                  graph_path = img_path + "/img/JSON"
                  g.generate_temporal_graph_frames(graph_path, img_path + "/img")

                  #g.generate_temporal_graph(graph_path)

                  #g.classify_images(img_path)
                  #graph_path = TEMP_DIR
                  #g.generate_temporal_graph_frames(graph_path, img_path)

                  #g.tg.to_plot(os.path.join(OUT_DIR, visual))

                  g.tg.to_text(os.path.join(OUT_DIR, text))
                  #if os.path.isdir(TEMP_DIR):
                  #    shutil.rmtree(TEMP_DIR)
                  if os.path.isfile(os.path.join(OUT_DIR,text)):
                      with open(os.path.join(OUT_DIR,text)) as f:
                          context = "".join(map(str,f.readlines()))
                          return str(context)
                  else:
                    continue

In [ ]:
import contextlib
import time
q_ids = QA['question_id'].unique()
for q in q_ids:
    if 'Sequence' in q:
      choices = 'Options: '
      que = QA.query("question_id == '"+q+"'")
      video_id = que['video_id'].values[0]
      question = que['question'].values[0]
      start = que['start'].values[0]
      end = que['end'].values[0]
      for choice in que['choices']:
          for c in choice:
              choices+=c['choice']
      if not os.path.isdir(directory_path+'/trim/'+q+video_id+'-opencv'):
        prompt = run_oic(q, video_id, question, choices, start, end)
        if prompt is not None:
          QA.loc[QA['question_id'] == q, 'OIC_context'] = prompt
          answer = run_gpt(prompt, question + choices)
          QA.loc[QA['question_id'] == q, 'OIC_answer'] = str(answer)
          print('OIC answer: {}'.format(answer))
          QA.to_csv('STAR_eval/question_answers.csv')








ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Which object is the person able to hold after closing the window? The book.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-a1974b775c7e>", line 18, in <cell line: 4>
    prompt = run_oic(q, video_id, question, choices, start, end)
  File "<ipython-input-10-d1ce66e75193>", line 6, in run_oic
    for video in os.scandir(directory_path):
OSError: [Errno 107] Transport endpoint is not connected: 'STAR_eval/Charades'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most 

In [ ]:
import shutil
shutil.rmtree('temp')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-235f3cf87a6a>", line 2, in <cell line: 2>
    shutil.rmtree('temp')
  File "/usr/lib/python3.10/shutil.py", line 715, in rmtree
    onerror(os.lstat, path, sys.exc_info())
  File "/usr/lib/python3.10/shutil.py", line 713, in rmtree
    orig_st = os.lstat(path)
OSError: [Errno 107] Transport endpoint is not connected: 'temp'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/p